In [ ]:
!pip install pymilvus


In [62]:
from pymilvus import connections

connections.connect(
  alias="default", 
  host='localhost', 
  port='19530'
)

In [63]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(uri = "http://localhost:19530")

In [15]:
from pymilvus import MilvusClient, DataType, FunctionType, Function

schema = MilvusClient.create_schema(auto_id = True, enable_dynamic_field=True,)

schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)

# 3.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=1536)
schema.add_field(field_name="name", datatype = DataType.VARCHAR, max_length = 300)
schema.add_field(field_name="mota", datatype = DataType.VARCHAR, max_length = 4000)
schema.add_field(field_name = "catalog", datatype=DataType.VARCHAR, max_length = 300)

{'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}, {'name': 'name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 300}}, {'name': 'mota', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4000}}, {'name': 'catalog', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 300}}], 'enable_dynamic_field': True}

In [49]:


index_params = client.prepare_index_params()
index_params.add_index(field_name= "embedding", index_type= "IVF_FLAT", 
                      metric_type="COSINE",
                       params = {
                           "nlist":32,
                           "nprobe": 4
                       },
                       mmap_enabled=True,
                       )
index_params.add_index(field_name = "name", index_type = "AUTOINDEX")
index_params.add_index(field_name = "mota", index_type = "AUTOINDEX")
index_params.add_index(field_name = "catalog", index_type="AUTOINDEX")

In [52]:
client.create_collection(
    collection_name="products",
    schema=schema, index_params = index_params
)

In [51]:
client.drop_collection(collection_name = "products")

In [ ]:
client.describe_collection(collection_name = "products")

In [ ]:
from pymilvus import MilvusClient

# Connect to Milvus
client = MilvusClient(uri="http://localhost:19530")

# Replace with your actual collection name
collection_name = "products"

# Get all data (pagination may be necessary for large datasets)
# You can retrieve specific fields, e.g., ["id", "name", "embedding"]
results = client.query(
    collection_name=collection_name,
    filter=None,  # no filter means get all
    output_fields=["*"],  # fetch all fields,
    limit =10
)

# Print all documents
for item in results:
    print(item)


In [34]:
client.drop_collection(collection_name = "products")

In [ ]:
client.describe_collection(collection_name = "products")

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv('./my_n8n/products.csv')

In [20]:
df.iloc[0]

Link sản phẩm          https://vifiba.vn/san-pham/bo-loc-nuoc-vifiba-...
Hình                   https://e-website-s3.s3.ap-southeast-1.amazona...
Tên                                  Bộ lọc nước VIFIBA x Sanbee Khử CLO
Giá gốc                                                        600,000 đ
Giá km                                                         429,000 đ
Độ giảm giá                                                     GIẢM 28%
Đánh giá                                                             5.0
Đã bán                                                        Đã bán 185
Nhóm sản phẩm                                               Vòi Lọc Nước
Thông số kỹ thuật      Thương hiệu: VIFIBA x Sanbee\nChất liệu vỏ: Nh...
Mô tả sản phẩm         Mô tả sản phẩm\n  ĐẶC ĐIỂM BỘ LỌC NƯỚC VIFIBA ...
HDSD URL                                               No matching title
HDSD TEXT                                                            NaN
Chính sách bảo hành    \nPhạm vị bảo hành:\n\n\nSản

In [29]:
df["Tên"][0]

'Bộ lọc nước VIFIBA x Sanbee Khử CLO'

In [ ]:
from pymilvus import connections, Collection
from openai import OpenAI
from pymilvus import MilvusClient

client = MilvusClient(uri="http://localhost:19530")

# OpenAI setup
tokenizer = OpenAI(api_key="")
# Query
query_text = "nồi lẩu"

# Step 1: Embed query using OpenAI text-embedding-3-small
response = tokenizer.embeddings.create(
    model="text-embedding-3-small",
    input=query_text
)

query_vector = response.data[0].embedding  # length 1536



In [ ]:
for i in range(len(df)):
    mota = f"""
Tên: {df['Tên'][i]}
Nhóm sản phẩm: {df['Nhóm sản phẩm'][i]}
Mô tả sản phẩm: {df["Mô tả sản phẩm"][i]}
Thông số kỹ thuật: {df["Thông số kỹ thuật"][i]}
    """
    name = df["Tên"][i]
    catalog = df['Nhóm sản phẩm'][i]
    embedding = tokenizer.embeddings.create(model = 'text-embedding-3-small', input = mota)
    print(str(name))
    result = {
        "name": name,
        "catalog": catalog,
        "mota": mota,
        "embedding" : embedding.data[0].embedding
    }
    client.insert(collection_name = "products", data = result)

In [ ]:
from pymilvus import connections, Collection
from openai import OpenAI
from pymilvus import MilvusClient

client = MilvusClient(uri="http://localhost:19530")

# OpenAI setup
tokenizer = OpenAI(api_key="")
# Query
query_text = "Tôi muốn mua nồi lẩu cho 2 anh em nhà tôi"

# Step 1: Embed query using OpenAI text-embedding-3-small
response = tokenizer.embeddings.create(
    model="text-embedding-3-small",
    input=query_text
)

query_vector = response.data[0].embedding  # length 1536



In [ ]:
results = client.search(
    collection_name = "products",
    data=[query_vector],
    anns_field="embedding",  # replace with your vector field name
    # param={"params": {"nprobe": 10}},
    limit=3,
    search_params={
        "params": {
            "radius": 0.4,
            # "range_filter": 0.6
        }
    },
    output_fields=["name", "catalog"]  # include any fields you want returned
)

# Step 3: Display results
hits = []
for i, hit in enumerate(results[0]):
    name = hit.get('entity','').get('name','')
    catalog = hit.get('entity','').get('catalog','')
    output = {
        "name":name,
        "category": catalog
    }
    hits.append(output)
print(hits)

[{'Tên': 'Nồi Lẩu 2 Ngăn KIDAI VIFIBA - Song Lẩu', 'Nhóm sản phẩm': 'Nồi Điện Đa Năng'}, {'Tên': 'Nồi Lẩu Hấp SAKI VIFIBA - Lẩu Hấp Vuông', 'Nhóm sản phẩm': 'Nồi Hấp Điện'}, {'Tên': 'Nồi Lẩu Hấp Điện Kachi Vifiba 15L - Bảo hành 25 Tháng', 'Nhóm sản phẩm': 'Nồi Điện Đa Năng'}]


In [2]:
!pip install boto3 pdf2image

  Using cached boto3-1.39.3-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 2.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 13.3 MB/s eta 0:00:00


In [6]:
!pip install poppler

ERROR: Could not find a version that satisfies the requirement poppler (from versions: none)
ERROR: No matching distribution found for poppler
